# Livrable 3 - Captionning d'images

|Auteurs|
|---|
|Frédéric SPATARO|
|Oscar PALISSOT|
|Djayan DEMAISON|
|Arnaud HITTINGER|
|Nicolas PELLEGRINI|

# 1. Introduction

## 1.1. Besoins du projet

L'objectif principal de ce projet est de développer un système automatique de génération de légendes pour des images. Cette tâche est cruciale pour l'entreprise TouNum, qui vise à étendre ses services de numérisation avec des fonctionnalités d'intelligence artificielle. Le "captioning" d'images, ou la génération automatique de légendes, permettra d'ajouter une valeur significative aux services existants, en facilitant la catégorisation et la recherche d'images dans les bases de données numérisées.

## 1.2. Dataset "MS COCO"

Pour atteindre cet objectif, nous utiliserons le dataset MS COCO (Microsoft Common Objects in Context), qui est largement reconnu dans la communauté de l'apprentissage automatique pour l'entraînement et l'évaluation des modèles de captioning d'images. Ce dataset contient plus de 200,000 images annotées avec environ 1.2 million de légendes, fournissant ainsi une ressource riche pour l'entraînement de notre modèle.

# 2. Installation et importation 

## 2.1. Installation des bibliothèques nécessaires

In [ ]:
!pip install tensorflow
!pip install -U scikit-learn scipy matplotlib
!pip install numpy
!pip install Pillow
!pip install tqdm

print("Installation terminée.")

## 2.2. Importation des bibliothèques nécessaires

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import collections
import random
import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
from tqdm import tqdm

import zipfile

print("Importation terminée.")

# 3. Chargement des données

## 3.1. Dézipage des données

On va décompresser les données stockées au format ZIP du dataset MS COCO.

In [ ]:
repertoire = './Train2014Zip'
dossier_dataset = './train2014'
if not os.path.exists(dossier_dataset):
    os.makedirs(dossier_dataset)

for fichier in os.listdir(repertoire):
    chembin_fichier = os.path.join(repertoire,fichier)
    if zipfile.is_zipfile(chemin_fichier):
        with zipfile.ZipFile(chemin_fichier, 'r') as zip_ref:
            zip_ref.extractall(dossier_dataset)
            print(f'Fichier {fichier} décompressé dans le dossier Dataset.')

print("Tous les fichiers ZIP ont été décompressés.")

## 3.2. Définition des chemins d'accès aux données

On définit les chemins d'accès aux :

* Fichier d'annotations
* Dossier des images du dataset à annoter

In [ ]:
# Chemin du fichier d'annotations
annotation_folder = os.path.abspath('.')+"/annotations/"
annotation_file = annotation_folder+"captions_train2014.json"

# Chemin du dossier contenant les images à annoter
image_folder = '/train2014/train2014/'
PATH = os.path.abspath('.') + image_folder

print("Fichier d'annotation : ", annotation_file)
print("Le chemin PATH : ", PATH)

## 3.3. Chargement des annotations et des instances

In [ ]:
with open(annotation_folder+"/captions_val2014.json", 'r') as f:
    annotations = json.load(f)

with open(annotation_folder+"instances_val2014.json", 'r') as f:
    instances = json.load(f)
    
print("Annotations et instances chargées.")

## 3.4. Organisation et préparation des annotations

On s'occupe de la préparation et de l'organisation des annotations textuelles associées à chaque image. Le but est d'avoir une structure organisée où chaque chemin d'accès à une image est associé à une liste de ses annotations. Cela permettra une utilisation efficace des données pendant le processus d'entraînement du modèle.

In [ ]:
# Lecture du fichier d'annotation
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

# Grouper toutes les annotations ayant le meme identifiant.
image_path_to_caption = collections.defaultdict(list)
for val in annotations['annotations']:
    # marquer le debut et la fin de chaque annotation
    caption = f"<start> {val['caption']} <end>"
    # L'identifiant d'une image fait partie de son chemin d'accès
    image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (val['image_id'])
    # Rajout du caption associé à image_path
    image_path_to_caption[image_path].append(caption)
    
print("Préparation terminée.")

## 3.5. Sélection d'un sous-ensemble pour l'entraînement

Les chemins d'accès à toutes les images sont mélangés aléatoirement. Cela garantit que la sélection d'images pour l'entraînement est variée et non biaisée. Seuls les chemins des 10 000 premières images sont sélectionnés pour l'entraînement après le mélange.

In [ ]:
# Prendre les premières images seulement
image_paths = list(image_path_to_caption.keys())
random.shuffle(image_paths)
train_image_paths = image_paths[:10000]

print("Nombres d'images utilisées :", len(train_image_paths))

## 3.6. Préparation des listes pour l'entraînement

On organise et on prépare les données nécessaires à l'entraînement du modèle de légendage.

In [ ]:
# Liste de toutes les annotations
train_captions = []
# Liste de tous les noms de fichiers des images dupliquées (en nombre d'annotations par image)
img_name_vector = []

for image_path in train_image_paths:
    caption_list = image_path_to_caption[image_path]
    # Rajout de caption_list dans train_captions
    train_captions.extend(caption_list)
    # Rajout de image_path dupliquée len(caption_list) fois
    img_name_vector.extend([image_path] * len(caption_list))
    
print("Préparation terminée.")

## 3.7. Visuation d'une image annotée

In [ ]:
print(len(train_captions), len(img_name_vector))
print(train_captions[0])
Image.open(img_name_vector[0])

# 4. Prétraitement des images et des annotations

## 4.1. Utilisation du modèle InceptionV3

On va s'appuyer la puissance des modèles pré-entraînés pour faciliter notre tâche de légendage d'images. Nous utilisons le modèle InceptionV3 de Google, largement reconnu pour ses performances en matière de classification d'images. Initialement formé sur le dataset ImageNet, ce modèle a appris à identifier et à extraire des caractéristiques essentielles de millions d'images. En utilisant ce modèle pré-entraîné, nous pouvons bénéficier de cette capacité d'extraction sans avoir à former un modèle à partir de zéro.

In [ ]:
# Telechargement du modèle InceptionV3 pré-entrainé avec la cassification sur ImageNet
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
# Creation d'une variable qui sera l'entrée du nouveau modèle de pre-traitement d'images
new_input = image_model.input
# récupérer la dernière couche caché qui contient l'image en representation compacte
hidden_layer = image_model.layers[-1].output

# Modèle qui calcule une representation dense des images avec InceptionV3
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

print("Modèle téléchargé.")

## 4.2. Traitement des images

La fonction `load_image` est conçue pour traiter et préparer une image pour la phase d'entraînement ou de prédiction. Elle commence par charger l'image depuis le chemin d'accès spécifié. Une fois l'image chargée, plusieurs étapes de traitement sont appliquées :

* L'image est d'abord décodée en format RGB, ce qui signifie qu'elle sera représentée en trois canaux : Rouge, Vert et Bleu.
* L'image est ensuite redimensionnée pour avoir une taille uniforme de 299×299 pixels (dimension attendue par le modèle).
* L'image est normalisée. Au lieu d'avoir des valeurs de pixels allant de 0 à 255, la normalisation ajuste ces valeurs pour qu'elles se situent entre -1 et 1.

In [ ]:
def load_image(image_path):
    """
    La fonction load_image a pour entrée le chemin d'une image et pour sortie un couple
    contenant l'image traitée ainsi que son chemin d'accès.
    La fonction load_image effectue les traitement suivant:
        1. Chargement du fichier correspondant au chemin d'accès image_path
        2. Décodage de l'image en RGB.
        3. Redimensionnement de l'image en taille (299, 299).
        4. Normalisation des pîxels de l'image entre -1 et 1
    """
    # Lecture du fichier image
    img = tf.io.read_file(image_path)
    # Decodage de l'image en RGB
    img = tf.image.decode_jpeg(img, channels=3)
    # Redimensionnement de l'image en taille (299, 299)
    img = tf.image.resize(img, (299, 299))
    # Normalisation des pîxels de l'image entre -1 et 1
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

print('load_image défini.')

## 4.3. Extraction des caractéristiques des images

On commence par créer un dataset d'images. Pour chaque batch d'images, on utilise le modèle InceptionV3 pour transformer chaque image en une version compacte appelée "représentation dense". Ces représentations sont ensuite sauvegardées pour une utilisation ultérieure, et le nom original de l'image est utilisé pour nommer le fichier de la représentation dense.

In [ ]:
from tqdm import tqdm

# Pré-traitement des images
# Prendre les noms des images
encode_train = sorted(set(img_name_vector))

# Creation d'une instance de "tf.data.Dataset" partant des noms des images 
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
# Division du données en batchs après application du pré-traitement fait par load_image
image_dataset = image_dataset.map(
    load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

# Parcourir le dataset batch par batch pour effectuez le pré-traitement d'InceptionV3
for img, path in tqdm(image_dataset):
    # Pré-traitement du batch (de taille (16,8,8,2048)) courant par InceptionV3 
    batch_features = image_features_extract_model(img)
    # Resize du batch de taille (16,8,8,2048) en taille (16,64,2048)
    batch_features = tf.reshape(batch_features,
                                (batch_features.shape[0], -1, batch_features.shape[3]))
    # Parcourir le batch courant et stocker le chemin ainsi que le batch avec np.save()
    for bf, p in zip(batch_features, path):
        path_of_feature = p.numpy().decode("utf-8")
        # (chemin de l'image associe a sa nouvelle representation , representation de l'image)
        np.save(
            path_of_feature, bf.numpy()
        )

## 4.4. Prétraitement des annotations

**Tokenisation et séquençage :**

Les annotations textuelles, qui décrivent le contenu des images, doivent être converties en un format que le modèle peut comprendre. Pour ce faire, on utilise un tokenizer. Le tokenizer transforme chaque mot des annotations en un numéro unique (token). Dans ce code, on limite le vocabulaire aux 5 000 mots les plus fréquents pour gérer la complexité.

Après avoir construit le vocabulaire à partir des annotations, un token spécial <pad> est ajouté. Ce token sera utilisé pour égaliser la longueur des séquences, car les modèles de traitement du langage naturel nécessitent des entrées de taille fixe.

**Padding des séquences :**
    
Après la tokenisation, chaque annotation est convertie en une séquence de tokens. Cependant, comme les annotations ont des longueurs variables, on utilise le padding pour s'assurer que toutes les séquences ont la même longueur, celle de l'annotation la plus longue.

**Sauvegarde du tokenizer :**
    
Le tokenizer, qui détient le mappage entre les mots et leurs tokens, est essentiel pour la suite du processus. Pour éviter de le recalculer à chaque fois, nous le sauvegardons sous forme de fichier JSON. Cela permettra de le recharger plus tard, par exemple lors de l'évaluation du modèle ou de son déploiement.

In [ ]:
# Trouver la taille maximale 
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

# Chosir les 5000 mots les plus frequents du vocabulaire
top_k = 5000
#La classe Tokenizer permet de faire du pre-traitement de texte pour reseau de neurones 
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
# Construit un vocabulaire en se basant sur la liste train_captions
tokenizer.fit_on_texts(train_captions)

# Créer le token qui sert à remplir les annotations pour egaliser leurs longueur
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

# Creation des vecteurs(liste de token entiers) à partir des annotations (liste de mots)
train_seqs = tokenizer.texts_to_sequences(train_captions)

# Remplir chaque vecteur à jusqu'à la longueur maximale des annotations
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

# Calcule la longueur maximale qui est utilisée pour stocker les poids d'attention 
# Elle servira plus tard pour l'affichage lors de l'évaluation
max_length = calc_max_length(train_seqs)

# Convert the tokenizer to a JSON string
tokenizer_json = tokenizer.to_json()

# Save the JSON string to a file
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
    
print('Annotations prétraités.')

# 5. Préparation et configuration des données pour l'entraînement

## 5.1 Formation du jeu d'entrainement et de test

On a opté pour une répartition de 80% des données pour l'entraînement et 20% pour le test.

In [ ]:
img_to_cap_vector = collections.defaultdict(list)
# Creation d'un dictionnaire associant les chemins des images avec (fichier .npy) aux annotationss
# Les images sont dupliquées car il y a plusieurs annotations par image
print(len(img_name_vector), len(cap_vector))
for img, cap in zip(img_name_vector, cap_vector):
    img_to_cap_vector[img].append(cap)

"""
Création des datasets de formation et de validation en utilisant 
un fractionnement 80-20 de manière aléatoire
"""
# Prendre les clés (noms des fichiers d'images traites), *celles-ci ne seront pas dupliquées*
img_keys = list(img_to_cap_vector.keys())
random.shuffle(img_keys) # Mélanger les clés
# Diviser des indices en entrainement et test
slice_index = int(len(img_keys)*0.8)
# divide into train and validation
img_name_train_keys, img_name_val_keys = img_keys[:slice_index], img_keys[slice_index:]

"""
Les jeux d'entrainement et de tests sont sous forme
de listes contenants les mappings :(image prétraitée ---> jeton d'annotation(mot) )
"""

# Boucle pour construire le jeu d'entrainement
img_name_train = []
cap_train = []
for imgt in img_name_train_keys:
    capt_len = len(img_to_cap_vector[imgt])
    # Duplication des images en le nombre d'annotations par image
    img_name_train.extend([imgt] * capt_len)
    cap_train.extend(img_to_cap_vector[imgt])

# Boucle pour construire le jeu de test
img_name_val = []
cap_val = []
for imgv in img_name_val_keys:
    capv_len = len(img_to_cap_vector[imgv])
    # Duplication des images en le nombre d'annotations par image
    img_name_val.extend([imgv] * capv_len)
    cap_val.extend(img_to_cap_vector[imgv])

len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

## 5.2 Configuration du dataset pour l'entraînement

**Paramétrage Initial :**

On définit plusieurs paramètres essentiels pour l'entraînement, tels que la taille du batch (`BATCH_SIZE`), la taille du buffer pour mélanger les données (`BUFFER_SIZE`), la dimension de l'embedding (`embedding_dim`), la taille de la couche cachée du RNN (`units`) et la taille du vocabulaire (`vocab_size`). De plus, on calcule le nombre d'étapes par époque en fonction de la taille du batch et du nombre d'images d'entraînement.

**Préparation du Dataset :**

L'étape suivante consiste à créer un dataset TensorFlow à partir des noms des fichiers d'images traitées et des annotations. Une fonction, `map_func`, est définie pour charger les fichiers numpy correspondant à chaque image et renvoyer le tensor de l'image avec son annotation.

Le dataset est ensuite configuré pour charger ces tensors en utilisant la fonction map_func. La méthode shuffle est utilisée pour mélanger les données, ce qui est essentiel pour garantir que le modèle ne mémorise pas l'ordre des données et généralise bien. Enfin, `prefetch` est utilisé pour améliorer les performances en préchargeant les données pendant l'entraînement.

In [ ]:
# N'hésitez pas à modifier ces paramètres en fonction de votre machine
BATCH_SIZE = 64 # taille du batch
BUFFER_SIZE = 1000 # taille du buffer pour melanger les donnes
embedding_dim = 256
units = 512 # Taille de la couche caché dans le RNN
vocab_size = top_k + 1
num_steps = len(img_name_train) // BATCH_SIZE

# La forme du vecteur extrait à partir d'InceptionV3 est (64, 2048)
# Les deux variables suivantes representent la forme de ce vecteur
features_shape = 2048
attention_features_shape = 64

# Fonction qui charge les fichiers numpy des images prétraitées
def map_func(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8')+'.npy')
    return img_tensor, cap

# Creation d'un dataset de "Tensor"s (sert à representer de grands dataset)
# Le dataset est cree a partir de "img_name_train" et "cap_train"
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# L'utilisation de map permet de charger les fichiers numpy (possiblement en parallèle)
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
    map_func, [item1, item2], [tf.float32, tf.int32]),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Melanger les donnees et les diviser en batchs
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

print('Dataset configuré.')

# 6. Le modèle

## 6.1. Encodeur CNN

L'encodeur CNN est conçu pour traiter les caractéristiques extraites des images par le modèle InceptionV3. Ces caractéristiques, qui sont une représentation dense de l'image, sont passées à travers cet encodeur pour être utilisées dans le modèle de légendage.

In [ ]:
class CNN_Encoder(tf.keras.Model):
    # Comme les images sont déjà prétraités par InceptionV3 est représenté sous forme compacte
    # L'encodeur CNN ne fera que transmettre ces caractéristiques à une couche dense
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # forme après fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

print('Encodeur CNN configuré.')

## 6.2. Mécanisme d'attention de Bahdanau

Ce modèle implémente le mécanisme d'attention de Bahdanau, qui est largement utilisé dans les tâches de traitement du langage naturel pour permettre au modèle de se "concentrer" sur différentes parties des données d'entrée lors de la génération de la sortie.

In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features(CNN_encoder output) forme == (batch_size, 64, embedding_dim)

        # forme de la couche cachée == (batch_size, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                             self.W2(hidden_with_time_axis)))

        # Cela vous donne un score non normalisé pour chaque caractéristique de l'image.
        score = self.V(attention_hidden_layer)

        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

print("Mécanisme d'attention configuré.")

## 6.3. Décodeur RNN

Ce décodeur utilise un RNN (spécifiquement une GRU) pour générer des légendes à partir des représentations denses des images. Il intègre également un mécanisme d'attention pour aider le RNN à se concentrer sur différentes parties de l'image à différentes étapes de la génération de la légende.

In [ ]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        #Couche dense qui aura pour entrée la sortie du GRU
        self.fc1 = tf.keras.layers.Dense(self.units)
        # Dernière couche dense
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        # L'attention est defini par un modèle a part
        context_vector, attention_weights = self.attention(features, hidden)
        # Passage du mot courant à la couche embedding
        x = self.embedding(x)
        # Concaténation
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # Passage du vecteur concaténé à la gru
        output, state = self.gru(x)

        # Couche dense
        y = self.fc1(output)

        y = tf.reshape(y, (-1, x.shape[2]))

        # Couche dense
        y = self.fc2(y)

        return y, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))
    
print('Décodeur RNN configuré.')

## 6.4. Combinaison de la la partie encodeur et décodeur 

In [ ]:
# Création de l'encodeur
encoder = CNN_Encoder(embedding_dim)
# Création du décodeur
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

print('Encodeur et décodeur combiné.')

## 6.5. Configuration de l'optimisation et de la fonction de loss

In [ ]:
# Optimiseur ADAM
optimizer = tf.keras.optimizers.Adam()
# La fonction de perte
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

print("Configuration de l'optimisation terminé.")

## 6.6. Mise en place du checkpoint

Pour garder la trace de notre apprentissage et la sauvegarder, on va utiliser la classe [`tf.train.Checkpoint`](https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint).


In [ ]:
checkpoint_path = os.path.abspath("checkpoints")
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

print("Checkpoint défini.")

## 6.7. Initialisation du checkpoint

Initialisation de l'époque de début d’entrainement dans `start_epoch`. La classe `tf.train.Checkpoint` permet de poursuivre l’entrainement là ou vous l’avez laissé s’il avait été interrompu auparavant.

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # Restaurer le dernier checkpoint dans checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)
    
print("Checkpoint initialisé.")

# 7. Entrainement et test :

## 7.1 Définition de l'entraînement du modèle

In [ ]:
loss_plot = []
@tf.function
def train_step(img_tensor, target):
    loss = 0

    # Initialisation de l'état caché pour chaque batch
    hidden = decoder.reset_state(batch_size=target.shape[0])

    # Initialiser l'entrée du décodeur
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

    with tf.GradientTape() as tape: # Offre la possibilité de calculer le gradient du loss
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # Prédiction des i'èmes mot du batch avec le décodeur
            predictions, hidden, _ = decoder(dec_input, features, hidden)
            loss += loss_function(target[:, i], predictions)

            # Le mot correct à l'étap i est donné en entrée à l'étape (i+1)
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

## 7.2 Entraînement du modèle

Le code global contenant la boucle d'entrainement est présenté ci-dessous. Cette boucle parcours le jeu de données d'entrainement batch par batch et entraine le réseaux avec ceux-ci.

In [ ]:
EPOCHS = 40

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
                epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # sauvegarde de la perte
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
        ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# Affichage de la courbe d'entrainement
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

## 7.3 Définition de la fonction de test du modèle

In [ ]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

# Fonction permettant la représentation de l'attention au niveau de l'image
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

## 7.4 Test du modèle

L'affichage de quelques exemples sur le résultat retourné par l'évaluation.

In [ ]:
# Affichage de quelques annotations dans le jeu de test
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
print(rid)
print(len(cap_val))
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)

In [ ]:
Image.open(img_name_val[rid])